<a href="https://colab.research.google.com/github/SuperPanww/courses_ML20/blob/master/hw3_panww.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!gdown --id '19CzXudqN58R3D-1G8KeFWk8UDQwlb8is' --output food-11.zip # 下載資料集
!unzip food-11.zip # 解壓縮

In [0]:
# Import需要的套件
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import time

In [0]:
def readfile(path, label):
  image_dir = sorted(os.listdir(path))
  x = np.zeros((len(image_dir),128,128,3),dtype=np.uint8)
  y = np.zeros((len(image_dir)),dtype=np.uint8)
  for i,file in enumerate(image_dir):
    img = cv2.imread(os.path.join(path, file))
    x[i,:] = cv2.resize(img,(128,128))
    if label:
      y[i] = int(file.split("_")[0])
  if label:
    return x,y
  else:
    return x

In [4]:
workspace_dir = "./food-11"
print('Reading data')
train_x,train_y = readfile(os.path.join(workspace_dir,'training'),True)
print("Size of training data = {}".format(len(train_x)))
val_x, val_y = readfile(os.path.join(workspace_dir,"validation"),True)
print("Size of validation data = {}".format(len(val_x)))
test_x = readfile(os.path.join(workspace_dir,"testing"),False)
print("Size of Testing data = {}".format(len(test_x)))

Reading data
Size of training data = 9866
Size of validation data = 3430
Size of Testing data = 3347


In [0]:
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),                                     
])

test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
])

class ImgDataset(Dataset):
  def __init__(self,x,y = None, transform = None):
    self.x = x
    self.y = y
    if y is not None:
      self.y = torch.LongTensor(y)
    self.transform = transform
  def __len__(self):
    return len(self.x)
  def __getitem__(self, index):
    X = self.x[index]
    if self.transform is not None:
      X = self.transform(X)
    if self.y is not None:
      Y = self.y[index]
      return X,Y
    else:
      return X

In [0]:
batch_size = 128
train_set = ImgDataset(train_x,train_y, train_transform)
val_set = ImgDataset(val_x,val_y,test_transform)
train_loader = DataLoader(train_set,batch_size = batch_size,shuffle=True)
val_loader = DataLoader(val_set,batch_size=batch_size,shuffle=False)

In [0]:
class Classifier(nn.Module):
  def __init__(self):
    super(Classifier, self).__init__()

    self.cnn = nn.Sequential(
        nn.Conv2d(3,64,3,1,1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(2,2,0),

        nn.Conv2d(64,128,3,1,1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(2,2,0),

        nn.Conv2d(128,256,3,1,1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(2,2,0),

        nn.Conv2d(256,512,3,1,1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.MaxPool2d(2,2,0),

        nn.Conv2d(512,512,3,1,1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.MaxPool2d(2,2,0),
    )
    self.fc = nn.Sequential(
        nn.Linear(512*4*4, 1024),
        nn.ReLU(),
        nn.Linear(1024, 512),
        nn.ReLU(),
        nn.Linear(512, 11)       
    )
  def forward(self, x):
    out = self.cnn(x)
    out = out.view(out.size()[0], -1)
    return self.fc(out)

In [13]:
model = Classifier().cuda()
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)
num_epoch = 30

for epoch in range(num_epoch):
  epoch_start_time = time.time()
  train_acc = 0.0
  train_loss = 0.0
  val_acc = 0.0
  val_loss = 0.0

  model.train()
  for i, data in enumerate(train_loader):
    optimizer.zero_grad()
    train_pred = model(data[0].cuda())
    batch_loss = loss(train_pred, data[1].cuda())
    batch_loss.backward()
    optimizer.step()

    train_acc += np.sum(np.argmax(train_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
    train_loss += batch_loss.item()

  model.eval()
  with torch.no_grad():
    for i, data in enumerate(val_loader):
      val_pred = model(data[0].cuda())
      batch_loss = loss(val_pred, data[1].cuda())

      val_acc += np.sum(np.argmax(val_pred.cpu().data.numpy(), axis = 1) == data[1].numpy())
      val_loss += batch_loss.item()
    
    print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f | Val Acc: %3.6f loss: %3.6f' % \
          (epoch + 1,num_epoch, time.time() - epoch_start_time, \
           train_acc/train_set.__len__(),train_loss/train_set.__len__(),val_acc/val_set.__len__(), val_loss/val_set.__len__()))


[001/030] 24.73 sec(s) Train Acc: 0.221265 Loss: 0.018186 | Val Acc: 0.237026 loss: 0.016879
[002/030] 24.73 sec(s) Train Acc: 0.324245 Loss: 0.015040 | Val Acc: 0.242566 loss: 0.021010
[003/030] 24.68 sec(s) Train Acc: 0.396919 Loss: 0.013714 | Val Acc: 0.136443 loss: 0.024250
[004/030] 24.69 sec(s) Train Acc: 0.439286 Loss: 0.012794 | Val Acc: 0.375219 loss: 0.015114
[005/030] 24.72 sec(s) Train Acc: 0.469086 Loss: 0.012199 | Val Acc: 0.280466 loss: 0.018427
[006/030] 24.72 sec(s) Train Acc: 0.505676 Loss: 0.011276 | Val Acc: 0.349854 loss: 0.017036
[007/030] 24.73 sec(s) Train Acc: 0.534259 Loss: 0.010724 | Val Acc: 0.202624 loss: 0.025183
[008/030] 24.74 sec(s) Train Acc: 0.548652 Loss: 0.010220 | Val Acc: 0.438776 loss: 0.014461
[009/030] 24.75 sec(s) Train Acc: 0.573586 Loss: 0.009655 | Val Acc: 0.412536 loss: 0.015859
[010/030] 24.76 sec(s) Train Acc: 0.602574 Loss: 0.009073 | Val Acc: 0.556851 loss: 0.010577
[011/030] 24.76 sec(s) Train Acc: 0.616359 Loss: 0.008731 | Val Acc: 0

In [0]:
train_val_x = np.concatenate((train_x,val_x), axis = 0)
train_val_y = np.concatenate((train_y,val_y), axis = 0)
train_val_set = ImgDataset(train_val_x, train_val_y, train_transform)
train_val_loader = DataLoader(train_val_set, batch_size = batch_size, shuffle = True)

In [19]:
model_best = Classifier().cuda()
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_best.parameters(), lr= 0.001)
num_epoch = 30

for epoch in range(num_epoch):
  epoch_start_time = time.time()
  train_acc = 0.0
  train_loss = 0.0
  
  model_best.train()
  for i,data in enumerate(train_val_loader):
    optimizer.zero_grad()
    train_pred = model_best(data[0].cuda())
    batch_loss = loss(train_pred, data[1].cuda())
    batch_loss.backward()
    optimizer.step()

    train_acc += np.sum(np.argmax(train_pred.cpu().data.numpy(), axis = 1) == data[1].numpy())
    train_loss += batch_loss.item()

  print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f' % \
        (epoch + 1, num_epoch, time.time()-epoch_start_time, \
         train_acc/train_val_set.__len__(),train_loss/train_val_set.__len__()))

[001/030] 29.46 sec(s) Train Acc: 0.247894 Loss: 0.017272
[002/030] 29.30 sec(s) Train Acc: 0.375526 Loss: 0.013933
[003/030] 29.21 sec(s) Train Acc: 0.440283 Loss: 0.012606
[004/030] 29.45 sec(s) Train Acc: 0.490298 Loss: 0.011576
[005/030] 29.41 sec(s) Train Acc: 0.524669 Loss: 0.010653
[006/030] 29.37 sec(s) Train Acc: 0.556258 Loss: 0.009957
[007/030] 29.37 sec(s) Train Acc: 0.596570 Loss: 0.009057
[008/030] 29.40 sec(s) Train Acc: 0.622819 Loss: 0.008550
[009/030] 29.40 sec(s) Train Acc: 0.647864 Loss: 0.007919
[010/030] 29.41 sec(s) Train Acc: 0.668096 Loss: 0.007455
[011/030] 29.39 sec(s) Train Acc: 0.685018 Loss: 0.007084
[012/030] 29.47 sec(s) Train Acc: 0.702843 Loss: 0.006673
[013/030] 29.39 sec(s) Train Acc: 0.718487 Loss: 0.006295
[014/030] 29.44 sec(s) Train Acc: 0.736613 Loss: 0.005961
[015/030] 29.44 sec(s) Train Acc: 0.751730 Loss: 0.005479
[016/030] 29.42 sec(s) Train Acc: 0.767900 Loss: 0.005222
[017/030] 29.45 sec(s) Train Acc: 0.783017 Loss: 0.004874
[018/030] 29.4

In [0]:
test_set = ImgDataset(test_x, transform = test_transform)
test_loader = DataLoader(test_set, batch_size = batch_size, shuffle= False)

In [0]:
model_best.eval()
prediction = []
with torch.no_grad():
  for i, data in enumerate(test_loader):
    test_pred = model_best(data.cuda())
    test_label = np.argmax(test_pred.cpu().data.numpy(), axis = 1)
    for y in test_label:
      prediction.append(y)

In [0]:
with open("predict.cas", 'w') as f:
  f.write('Id,Category\n')
  for i ,y in enumerate(prediction):
    f.write('{},{}\n'.format(i,y))